In [1]:
import sys
import os
sys.path.insert(0, '../')
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import oandapyV20.endpoints.instruments as instruments
import plotly.graph_objects as go
from src.fetch_historical_data import FetchHistoricalData
from src.q_learning import QLearningTrader
from src.utils import calculate_indicators
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
token = os.getenv("OANDA_ACCESS_TOKEN")

In [3]:
data = FetchHistoricalData('USD_JPY', 'M1', token, 5000).fetch_and_process_data()
data = calculate_indicators(data).dropna(inplace=False)

In [4]:
# train test split by index
train = data.iloc[:int(0.8*len(data))]
test = data.iloc[int(0.8*len(data)):]

In [5]:
qtrader = QLearningTrader(
            num_actions=3,
            num_features=11,
            learning_rate=0.01,
            discount_factor=0.9,
            exploration_prob=0.1,
        )

In [6]:
train

,High,Close,Low,Open,SMA,RSI,MACD,%K,%D,resistance,support
Time,,,,,,,,,,,
2024-04-09 17:22:00+08:00,151.868,151.856,151.856,151.866,151.8630,11.656240,-0.001530,8.333333,31.944444,151.865000,151.861000
2024-04-09 17:23:00+08:00,151.858,151.854,151.853,151.856,151.8606,9.399894,-0.002345,5.882353,20.016340,151.863188,151.858012
2024-04-09 17:24:00+08:00,151.866,151.856,151.855,151.855,151.8590,29.787015,-0.002361,20.000000,11.405229,151.861550,151.856450
2024-04-09 17:25:00+08:00,151.861,151.858,151.854,151.858,151.8578,47.505673,-0.001933,33.333333,19.738562,151.859933,151.855667
2024-04-09 17:26:00+08:00,151.868,151.864,151.858,151.860,151.8576,75.419417,-0.000621,73.333333,42.222222,151.859524,151.855676
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-12 12:16:00+08:00,153.229,153.226,153.225,153.225,153.2248,85.974243,0.002972,57.142857,73.214286,153.226142,153.223458
2024-04-12 12:17:00+08:00,153.226,153.224,153.222,153.226,153.2256,45.736219,0.002478,33.333333,50.992063,153.226047,153.225153
2024-04-12 12:18:00+08:00,153.228,153.226,153.224,153.224,153.2256,68.118317,0.002458,40.000000,43.492063,153.226047,153.225153


In [7]:
actions, cumulative_rewards = qtrader.train(train)

2024-04-14 16:38:03.990 | INFO     | src.q_learning:train:106 - Training the Q-learning model...


State: 
151.856, Action: 0, Reward: -1.3170371931179065e-05, Updated Q-value: None, Cumulative reward: -1.3170371931179065e-05
State: 
151.854, Action: 1, Reward: -1.3170545392160415e-05, Updated Q-value: -1.3170545392160414e-07, Cumulative reward: -2.634091732333948e-05
State: 
151.856, Action: 0, Reward: 1.3170371931366227e-05, Updated Q-value: 1.3170371931366227e-07, Cumulative reward: -1.3170545391973253e-05
State: 
151.858, Action: 1, Reward: -3.9510595424674544e-05, Updated Q-value: -3.939206207729225e-07, Cumulative reward: -5.26811408166478e-05
State: 
151.864, Action: 1, Reward: 0.0006453142285203418, Updated Q-value: 6.5847143007977666e-06, Cumulative reward: 0.000592633087703694
State: 
151.766, Action: 1, Reward: -0.00013178182201553862, Updated Q-value: 5.260311366341582e-06, Cumulative reward: 0.00046085126568815536
State: 
151.786, Action: 1, Reward: -7.247045182029753e-05, Updated Q-value: -7.247045182029752e-07, Cumulative reward: 0.00038838081386785784
State: 
151.797

2024-04-14 16:38:04.479 | INFO     | src.q_learning:train:145 - Training complete.


Final Q-table:
[[-3.07719792e-06  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.90457552e-05  0.00000000e+00]
 [-1.56217685e-06  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.70482040e-06  0.00000000e+00]
 [ 3.03465706e-06  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  3.30306276e-05  0.00000000e+00]]


In [8]:
train_plot = train.copy()
train_plot['Action'] = actions + [np.nan]
train_plot['Cumulative Rewards'] = cumulative_rewards

In [9]:
train_plot.head()

,High,Close,Low,Open,SMA,RSI,MACD,%K,%D,resistance,support,Action,Cumulative Rewards
Time,,,,,,,,,,,,,
2024-04-09 17:22:00+08:00,151.868,151.856,151.856,151.866,151.8630,11.656240,-0.001530,8.333333,31.944444,151.865000,151.861000,0.0,NaN
2024-04-09 17:23:00+08:00,151.858,151.854,151.853,151.856,151.8606,9.399894,-0.002345,5.882353,20.016340,151.863188,151.858012,1.0,-0.000013
2024-04-09 17:24:00+08:00,151.866,151.856,151.855,151.855,151.8590,29.787015,-0.002361,20.000000,11.405229,151.861550,151.856450,0.0,-0.000026
2024-04-09 17:25:00+08:00,151.861,151.858,151.854,151.858,151.8578,47.505673,-0.001933,33.333333,19.738562,151.859933,151.855667,1.0,-0.000013
2024-04-09 17:26:00+08:00,151.868,151.864,151.858,151.860,151.8576,75.419417,-0.000621,73.333333,42.222222,151.859524,151.855676,1.0,-0.000053


In [10]:
fig = go.Figure(data=[go.Candlestick(x=train.index,
                open=train_plot["Open"],
                high=train_plot["High"],
                low=train_plot["Low"],
                close=train_plot["Close"])])

fig.update_layout(
    title="Training data",
    xaxis=dict(
        rangeslider=dict(
            visible=False
        )
    )
)

fig.show()

In [11]:
action_dict = {0: 'Buy', 1: 'Sell', 2: 'Hold'}
train_plot['Action'] = train_plot['Action'].map(action_dict)

fig = go.Figure()

# Add cumulative reward trace
fig.add_trace(go.Scatter(x=train_plot.index, y=train_plot["Cumulative Rewards"], mode='lines', name='Cumulative Rewards', line=dict(width=4)))

# Add action traces
for action in train_plot["Action"].unique():
    mask = train_plot["Action"] == action
    fig.add_trace(go.Scatter(x=train_plot[mask].index, y=train_plot[mask]["Cumulative Rewards"], mode='markers', name=f'Action: {action}', marker=dict(size=6)))

fig.update_layout(title="Cumulative rewards for training set")
fig.show()

In [12]:
for idx, row in test.iterrows():
    new_data_df = pd.DataFrame(row).T
    qtrader.update(train, new_data_df)
    train =  pd.concat([train, new_data_df], ignore_index=True)

Hold signal detected.
State: 
High          153.227000
Close         153.224000
Low           153.218000
Open          153.223000
SMA           153.224400
RSI            50.837398
MACD            0.001366
%K             50.000000
%D             30.000000
resistance    153.225237
support       153.223563
Name: 2024-04-12 12:20:00+08:00, dtype: float64, Action: 2, Reward: 6.526392732240216e-06, Updated Q-value: 3.096886327556867e-06, Cumulative reward: 6.526392732240216e-06
Hold signal detected.
State: 
High          153.225000
Close         153.225000
Low           153.222000
Open          153.224000
SMA           153.224200
RSI            59.699023
MACD            0.001376
%K             58.333333
%D             36.111111
resistance    153.224942
support       153.223458
Name: 3995, dtype: float64, Action: 2, Reward: -1.3052700277246716e-05, Updated Q-value: 2.963262438456843e-06, Cumulative reward: -1.3052700277246716e-05
Hold signal detected.
State: 
High          153.225000
Close   